# Jordan Taylor

# Week 10 Homework

>Parks and green spaces provide a variety of health benefits by promoting physical activity, connection to nature and opportunities for community engagement. Nature-based programs can even be prescribed by health care providers as part of alternative, cost-effective treatment plans.

>In a paper recently published in the North Carolina Medical Journal, Lincoln Larson and Aaron Hipp, associate professors of parks, recreation and tourism management at NC State, provide an overview of how parks and green spaces can help alleviate contemporary health challenges.

[Source](https://cnr.ncsu.edu/news/2022/04/parks-green-spaces-improve-health/)

## 10.1 Finding the nearest parks to schools in Los Angeles County
You are given information on the point locations of schools in the LA Unified Schoo School District (LAUSD) (``` data/Schools_LAUSD.geojson```) and polygons of LA city parks (```data/Recreation_and_Parks.geojson ```)

1. Load the school and park data. For calcuating distances, what should you check and potentially change? 
2. Find the nearest park to each school in the LAUSD. Capture the  ``` Name ``` of the closest park. Note, this is not as straight forward as just copying and pasting from section-02 week 10! You actually have to think logically what columns are needed (centroids or geometries? what is the target dataframe?). Think carefully! 
3. Create an interactive map which plots the LAUSD schools locations and has a ``` Tooltip ``` capability of showing the ``` FULLNAME ``` of the school (using alias of ``` School Name ```) and the name of the closest park using the alias of ``` Nearest Park Name ```.
4. Save your map as a .html file, upload to your GitHub, and provide a link where anyone can see the final product. Paste this link into the notebook you submit for the assignment. 

In [25]:
import rasterio
from rasterio.plot import show
from rasterstats import zonal_stats
import osmnx as ox
import geopandas as gpd
import os
import matplotlib.pyplot as plt
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points
%matplotlib inline

#additional packages
import folium
import pandas as pd
from pyproj import crs
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt

In [3]:
school_path = 'data/Schools_LAUSD.geojson'

In [4]:
park_path = 'data/Recreation_and_Parks.geojson'

In [5]:
schools = gpd.read_file(school_path)
parks = gpd.read_file(park_path)

In [6]:
schools.head(3)

,ID,LOCN,MPD_NAME,ADDRESS,CITY,ZIP,MPD_TYPE,MAP_TYPE,LABEL,MAP_DESC,MPD_DESC,LD,FULLNAME,CDSCODE,CHARTER,MAP_TYPE_MAPPING,TOOLTIP,NLA_URL,geometry
0,1,2835,VISTA DEL VALLE AC DL,12411 N BROMONT AVE,LOS ANGELES,91340,DS,ED,Vista del Valle Ac DL,Spanish Dual Language Program,Dual Language Program,0,Vista del Valle Dual Language Academy Dual Lan...,,,ES,Name: Vista del Valle Dual Language Academy Du...,navigatela/reports/lausd_school_report.cfm?PK=1,POINT (-118.42169 34.29553)
1,2,2820,PARMELEE AVE EL DL SP,1338 E 76TH PL,LOS ANGELES,90001,DS,ED,Parmelee Ave El DL Sp,Spanish Dual Language Program,Dual Language Program,0,Parmelee Avenue Elementary Dual Language Spanish,,,ES,Name: Parmelee Avenue Elementary Dual Language...,navigatela/reports/lausd_school_report.cfm?PK=2,POINT (-118.25099 33.97006)
2,3,2773,OCHOA LC DLC SP,5057 LIVE OAK ST,CUDAHY,90201,DS,ED,Ochoa LC DLC Sp,Spanish Dual Language Program,Dual Language Program,0,Ellen Ochoa Learning Center Dual Language Spanish,,,ES,Name: Ellen Ochoa Learning Center Dual Languag...,navigatela/reports/lausd_school_report.cfm?PK=3,POINT (-118.17660 33.96793)


In [7]:
parks.head(3)

,OBJECTID,Area_Acres,Park_ID,CD_Real,Address,Region,Name,Park_Class,CD_NEW,TOOLTIP,ACRES,geometry
0,1,8.433364,512,03,23751 Ingomar St,V,Lazy J Ranch Park,N,12,Name: Lazy J Ranch Park\nLocation: 23751 Ingom...,8.4333635,"POLYGON ((-118.64451 34.21238, -118.64451 34.2..."
1,2,1.387374,883,15,Adjacent to Angels Gate Park at 3400 Gaffey St...,P,Lookout Point Park,N,15,Name: Lookout Point Park\nLocation: Adjacent t...,1.38737377,"POLYGON ((-118.29192 33.71197, -118.29192 33.7..."
2,3,3.173861,320,15,1464 E 109th Street,P,109th Street Recreation Center,N,15,Name: 109th Street Recreation Center\nLocation...,3.17386107,"POLYGON ((-118.24925 33.93590, -118.24917 33.9..."


In [8]:
print(schools.crs)
print(parks.crs)

epsg:4326
epsg:4326


In [9]:
schools = schools.to_crs(3067)
parks = parks.to_crs(3067)

#schools = schools.to_crs(epsg=3067)
#parks = parks.to_crs(epsg=3067)

In [10]:
print(schools.crs)
print(parks.crs)

epsg:3067
epsg:3067


In [11]:
selected_schools = schools[['ID', 'FULLNAME', 'geometry']]

In [12]:
selected_schools.head(10)

,ID,FULLNAME,geometry
0,1,Vista del Valle Dual Language Academy Dual Lan...,POINT (-2746611.448 15605456.467)
1,2,Parmelee Avenue Elementary Dual Language Spanish,POINT (-2778068.707 15637139.760)
2,3,Ellen Ochoa Learning Center Dual Language Spanish,POINT (-2785440.214 15634562.135)
3,4,Wilton Place Elementary Dual Language Korean,POINT (-2767751.543 15629201.571)
4,5,Miles Avenue Elementary Dual Language Spanish,POINT (-2780972.975 15635000.113)
5,6,Micheltorena Street Elementary Dual Language S...,POINT (-2770251.937 15624104.956)
6,7,Mayberry Street Elementary Dual Language Spanish,POINT (-2771518.242 15624298.835)
7,8,Gardena Elementary Dual Language Spanish,POINT (-2778574.957 15648793.103)
8,9,Judith F Baca Arts Academy Dual Language Spanish,POINT (-2779186.972 15638636.850)
9,10,Charles H Kim Elementary School Transitional B...,POINT (-2767923.102 15627557.220)


In [16]:
selected_parks = parks[[ 'OBJECTID','Name','Address','geometry']].reset_index()
selected_parks['centroid'] = selected_parks.centroid
parks_df = selected_parks[['OBJECTID','Name', 'Address', 'geometry','centroid']].reset_index()

In [17]:
parks_df.head()

,index,OBJECTID,Name,Address,geometry,centroid
0,0,1,Lazy J Ranch Park,23751 Ingomar St,"POLYGON ((-2728718.751 15623568.405, -2728717....",POINT (-2728622.720 15623506.109)
1,1,2,Lookout Point Park,Adjacent to Angels Gate Park at 3400 Gaffey St...,"POLYGON ((-2785774.485 15669003.142, -2785771....",POINT (-2785729.994 15668953.079)
2,2,3,109th Street Recreation Center,1464 E 109th Street,"POLYGON ((-2779794.504 15641083.407, -2779802....",POINT (-2779840.029 15641008.287)
3,3,4,Angeles Mesa Park,4726 8th Avenue,"POLYGON ((-2769444.289 15636423.338, -2769438....",POINT (-2769419.491 15636423.769)
4,4,5,Julian C. Dixon Park,4800 S. Hoover St,"POLYGON ((-2773258.861 15635123.968, -2773274....",POINT (-2773249.093 15635077.300)


In [18]:
schools_df = selected_schools.copy()

In [19]:
def get_nearest_values(row, other_gdf, point_column='geometry', value_column="geometry"):
    """Find the nearest point and return the corresponding value from specified value column."""
    
    # Create an union of the other GeoDataFrame's geometries:
    other_points = other_gdf["centroid"].unary_union
    
    # Find the nearest points:
    nearest_geoms = nearest_points(row[point_column], other_points)
    
    # Get corresponding values from the other df:
    nearest_data = other_gdf.loc[other_gdf["centroid"] == nearest_geoms[1]]
    
    nearest_value = nearest_data[value_column].values[0]
    
    return nearest_value

In [21]:
schools_df["nearest_loc"] = schools_df.apply(get_nearest_values, 
                                             other_gdf=parks_df, point_column="geometry",value_column="Name", axis=1)

In [22]:
schools_df.head()

,ID,FULLNAME,geometry,nearest_loc
0,1,Vista del Valle Dual Language Academy Dual Lan...,POINT (-2746611.448 15605456.467),Hubert H. Humphrey Memorial Park
1,2,Parmelee Avenue Elementary Dual Language Spanish,POINT (-2778068.707 15637139.760),Fremont High School Pool
2,3,Ellen Ochoa Learning Center Dual Language Spanish,POINT (-2785440.214 15634562.135),Jordan Downs Recreation Center
3,4,Wilton Place Elementary Dual Language Korean,POINT (-2767751.543 15629201.571),Harold A. Henry Park
4,5,Miles Avenue Elementary Dual Language Spanish,POINT (-2780972.975 15635000.113),Pueblo del Rio Recreation Center


In [26]:
schools_df = schools_df.to_crs(epsg=4326)

In [34]:
m = folium.Map(location=[33.9710, -118.417], tiles = 'CartoDB dark_matter', zoom_start=10, control_scale=True)
folium.features.GeoJson(schools_df,  
                        name='LA County Schools and Parks',
                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
                        tooltip=folium.features.GeoJsonTooltip(fields=['FULLNAME','nearest_loc'],
                        aliases = ['School Name', 'Nearest Park Name'],
                        labels=True,
                        sticky=False
                                    )
                       ).add_to(m)
folium.LayerControl().add_to(m)

m

## Live link to my map:

### https://jordantaylor44.github.io/geospatial-2023/Taylor-week10-schools-map.html